In [16]:
import numpy as np
import pandas as pd
import sklearn
import gensim
import json
import re

d_adr = "data/"
train = pd.read_csv(d_adr + "train.csv")
test = pd.read_csv(d_adr + "test.csv")
chunk = [[],[],[],[],[],[]]
chunk[0] = open(d_adr + "json/chunk0.json").read().split("\n")
chunk[1] = open(d_adr + "json/chunk1.json").read().split("\n")
chunk[2] = open(d_adr + "json/chunk2.json").read().split("\n")
chunk[3] = open(d_adr + "json/chunk3.json").read().split("\n")
chunk[4] = open(d_adr + "json/chunk4.json").read().split("\n")
chunk[5] = open(d_adr + "json/chunk5.json").read().split("\n")

In [18]:
data_id = []
data_images = []
data_links = []
data_title = []
data_text = []
data_text_sentence = []
data_special = []
cnt=0
for i in range(0,5):
    for st in chunk[i]:
        if st == "":continue
        if cnt%10000 == 0 : print (cnt)
        #if cnt>100 : break
        data = json.loads(st)
        id_s = ""
        for temp in data['id']:
            id_s+=temp
        data_id.append(id_s)
        data_images.append(len(data['images']))
        data_links.append(len(data['links']))
        title_s = ""
        for temp in data['title']:
            title_s+=temp
        data_title.append(title_s)
        data_text.append(len(data['text']))
        sentence = ""
        for temp in data['text']:
            sentence+="" + temp
        data_special.append(len(re.findall("[^a-zA-Z0-9_ ]",sentence)))
        data_text_sentence.append(sentence.strip())
        cnt+=1
        
train_all = pd.DataFrame({"id":data_id,
                          "images":data_images,
                          "links":data_links,
                          "title":data_title,
                          "text":data_text,
                          "sentence_text":data_text_sentence,
                          "special_len":data_special
                         })
train_all['file']=train_all['id'].apply(lambda x: x + '_raw_html.txt')
train_labeled = pd.merge(train,train_all,how='left',on='file')
train_labeled

data_id = []
data_images = []
data_links = []
data_title = []
data_text = []
data_text_sentence = []
data_special = []
cnt=0
i=5
for st in chunk[i]:
    if st == "":continue
    if cnt%10000 == 0 : print (cnt)
    #if cnt>100 : break
    data = json.loads(st)
    id_s = ""
    for temp in data['id']:
        id_s+=temp
    data_id.append(id_s)
    data_images.append(len(data['images']))
    data_links.append(len(data['links']))
    title_s = ""
    for temp in data['title']:
        title_s+=temp
    data_title.append(title_s)
    data_text.append(len(data['text']))
    sentence = ""
    for temp in data['text']:
        sentence+="" + temp
    data_special.append(len(re.findall("[^a-zA-Z0-9_ ]",sentence)))
    data_text_sentence.append(sentence.strip())
    cnt+=1

sample_all = pd.DataFrame({"id":data_id,
                          "images":data_images,
                          "links":data_links,
                          "title":data_title,
                          "text":data_text,
                          "sentence_text":data_text_sentence,
                           "special_len":data_special
                         })
sample_all['file']=sample_all['id'].apply(lambda x: x + '_raw_html.txt')
test_labeled = pd.merge(test,sample_all,how='left',on='file')

train=[]
train_all=[]
test=[]
sample_all=[]
chunk = []
data_id = []
data_images = []
data_links = []
data_title = []
data_text = []
data_text_sentence = []
cnt=0


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
0
10000
20000
30000
40000
50000
60000


In [13]:
from nltk.corpus import stopwords
from nltk.stem.porter import *
def text_to_words(review):
    stemmer = PorterStemmer()
    letters_only = re.sub("[^a-zA-Z]"," ",review)
    
    words = letters_only.split()
    
    stops = set(stopwords.words("english"))
    
    meaningful_words = [w for w in words if not w in stops]
    
    singles = [stemmer.stem(w) for w in meaningful_words]
    
    return len(singles),(" ".join(singles))
    #return len(meaningful_words),(" ".join(meaningful_words))

clean_train_text = []
clean_len=[]

num_train = train_labeled["sentence_text"].size

for i in range(0,num_train):
    if i % 10000 == 0 : print (i)
    #if i > 100 : break
    length, clean = text_to_words(train_labeled["sentence_text"][i])
    #clean = text_to_words(train_labeled["sentence_text"][i])
    clean_len.append(length)
    clean_train_text.append(clean)

train_labeled["sentence_text"]=clean_train_text
train_labeled["text_len"]=clean_len
clean_train_text=[]
    
clean_test_text = []
clean_len=[]

num_test = test_labeled["sentence_text"].size

for i in range(0,num_test):
    if i % 10000 == 0 : print (i)
    #if i > 100 : break
    length, clean = text_to_words(test_labeled["sentence_text"][i])
    #clean = text_to_words(test_labeled["sentence_text"][i])
    clean_len.append(length)
    clean_test_text.append(clean)

test_labeled["sentence_text"]=clean_test_text
test_labeled["text_len"]=clean_len
clean_test_text=[]
clean_len=[]

NameError: name 'train_labeled' is not defined

In [14]:
from gensim import corpora, models
from gensim.corpora import TextCorpus, MmCorpus, Dictionary
from nltk.corpus import stopwords
from optparse import OptionParser

dictionary = corpora.Dictionary(prune_at=150)

train_labeled = train_labeled.loc[0:100]
test_labeled = test_labeled.loc[0:100]
data = train_labeled["sentence_text"]
data = data.apply(lambda x:x.split(" "))

dictionary = corpora.Dictionary(data)

corpus = [dictionary.doc2bow(text) for text in data]

lda = gensim.models.ldamodel.LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=1000, chunksize=1000, passes=1)

#print max(lda.get_document_topics(corpus[0]),key=lambda item:item[1])
num = len(train_labeled)
df = pd.DataFrame()
for i in range(0,num):
    temp = pd.DataFrame(lda.get_document_topics(corpus[i],minimum_probability=0))
    df = df.append(temp.T.loc[1],ignore_index=True)

train = pd.concat([train_labeled,df],axis=1)
del train_labeled
del df

data = test_labeled["sentence_text"]
data = data.apply(lambda x:x.split(" "))
corpus = [dictionary.doc2bow(text) for text in data]

num = len(test_labeled)
df = pd.DataFrame()
for i in range(0,num):
    temp = pd.DataFrame(lda.get_document_topics(corpus[i],minimum_probability=0))
    df = df.append(temp.T.loc[1],ignore_index=True)
    
test = pd.concat([test_labeled,df],axis=1)
del test_labeled
del df

train

NameError: name 'train_labeled' is not defined

In [11]:
pre = list(train)
pre.remove('id')
pre.remove('title')
pre.remove('text')
pre.remove('sentence_text')

train[pre].to_csv(d_adr + "train_labeled.csv")
test[pre].to_csv(d_adr + "test_labeled.csv")

In [34]:
train_labeled["id"].isnull()[(train_labeled["id"].isnull() == True)]
test_labeled["id"].isnull()[(test_labeled["id"].isnull() == True)]
test_labeled.iloc[16369]

file             999839_raw_html.txt
sponsored                          0
id                               NaN
images                           NaN
links                            NaN
sentence_text                    NaN
special_len                      NaN
text                             NaN
title                            NaN
Name: 16369, dtype: object